task name: NormalizeMeanVariance
task description: Implement the NormalizeMeanVariance example using ML.NET

task name: NormalizeMeanVariance
task description: Implement the NormalizeMeanVariance example using ML.NET

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using static Microsoft.ML.Transforms.NormalizingTransformer;

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using static Microsoft.ML.Transforms.NormalizingTransformer;

In [ ]:
var mlContext = new MLContext();
public class DataPoint
{
    [VectorType(4)]
    public float[] Features { get; set; }
}

In [ ]:
var mlContext = new MLContext();
public class DataPoint
{
    [VectorType(4)]
    public float[] Features { get; set; }
}

In [ ]:
var samples = new List<DataPoint>()
{
    new DataPoint(){ Features = new float[4] { 1, 1, 3, 0} },
    new DataPoint(){ Features = new float[4] { 2, 2, 2, 0} },
    new DataPoint(){ Features = new float[4] { 0, 0, 1, 0} },
    new DataPoint(){ Features = new float[4] {-1,-1,-1, 1} }
};
var data = mlContext.Data.LoadFromEnumerable(samples);

In [ ]:
var samples = new List<DataPoint>()
{
    new DataPoint(){ Features = new float[4] { 1, 1, 3, 0} },
    new DataPoint(){ Features = new float[4] { 2, 2, 2, 0} },
    new DataPoint(){ Features = new float[4] { 0, 0, 1, 0} },
    new DataPoint(){ Features = new float[4] {-1,-1,-1, 1} }
};
var data = mlContext.Data.LoadFromEnumerable(samples);

In [ ]:
var normalize = mlContext.Transforms.NormalizeMeanVariance("Features", useCdf: true);
var normalizeNoCdf = mlContext.Transforms.NormalizeMeanVariance("Features", useCdf: false);

In [ ]:
var normalizeTransform = normalize.Fit(data);
var transformedData = normalizeTransform.Transform(data);
var normalizeNoCdfTransform = normalizeNoCdf.Fit(data);
var noCdfData = normalizeNoCdfTransform.Transform(data);

In [ ]:
var column = transformedData.GetColumn<float[]>("Features").ToArray();
foreach (var row in column)
    Console.WriteLine(string.Join(", ", row.Select(x => x.ToString("f4"))));
var columnFixZero = noCdfData.GetColumn<float[]>("Features").ToArray();
foreach (var row in columnFixZero)
    Console.WriteLine(string.Join(", ", row.Select(x => x.ToString("f4"))));

In [ ]:
// step9: get transformation parameters and print them
var transformParams = normalizeTransform?.GetNormalizerModelParameters(0) as CdfNormalizerModelParameters<float[]>;
var noCdfParams = normalizeNoCdfTransform?.GetNormalizerModelParameters(0) as AffineNormalizerModelParameters<float[]>;

if (transformParams != null && noCdfParams != null)
{
    Console.WriteLine($"The 1-index value in resulting array would be produced by: y = 0.5 * (1 + ERF((x - {transformParams.Mean[1]}) / ({transformParams.StandardDeviation[1]} * Math.Sqrt(2))))");
    var offset = noCdfParams.Offset.Length == 0 ? 0 : noCdfParams.Offset[1];
    var scale = noCdfParams.Scale[1];
    Console.WriteLine($"Values for slot 1 would be transformed by applying y = (x - ({offset})) * {scale}");
}